In [1]:
%matplotlib notebook

In [2]:
import pandas as pd
import gmaps
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from pprint import pprint

import sys
from config import gkey

gmaps.configure(api_key=gkey)

In [3]:
dfmain = pd.read_csv("FactData_Jun2018.txt")
dfloc = pd.read_csv("DTloc.txt")
dfmain.head()
dfloc.head(100)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LOCTYP,LOCTYPT,LOC,LOCT
0,1,United States,01,01-ALABAMA
1,1,United States,02,02-ALASKA
2,1,United States,04,04-ARIZONA
3,1,United States,05,05-ARKANSAS
4,1,United States,06,06-CALIFORNIA
5,1,United States,08,08-COLORADO
6,1,United States,09,09-CONNECTICUT
7,1,United States,10,10-DELAWARE
8,1,United States,11,11-DISTRICT OF COLUMBIA
9,1,United States,12,12-FLORIDA


In [4]:
dfcombo = pd.merge(dfloc, dfmain, on='LOC')
dfcombo.dropna()
dfcombo.head()


,LOCTYP,LOCTYPT,LOC,LOCT,AGYSUB,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,...,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,1,United States,01,01-ALABAMA,AF06,J,17,13,H,0511,...,K,XXXX,8,10,F,1,201806,1,113428.0,29.3
1,1,United States,01,01-ALABAMA,AF06,E,17,13,D,0511,...,I,XXXX,8,10,F,1,201806,1,93069.0,9.3
2,1,United States,01,01-ALABAMA,AF06,I,13,14,H,0511,...,M,XXXX,2,10,F,1,201806,1,130601.0,29.3
3,1,United States,01,01-ALABAMA,AF06,K,13,13,H,0511,...,K,XXXX,8,10,F,1,201806,1,110520.0,29.1
4,1,United States,01,01-ALABAMA,AF0J,H,4,6,D,0318,...,D,XXXX,8,10,F,1,201806,1,43424.0,7.7


In [5]:
locations = dfcombo[["LOCTYPT", "LOCT"]]
locations = locations.rename(columns={"LOCTYPT": "Country", "LOCT": "State"})

mask1 = locations["Country"] == "United States"
mask2 = locations["State"] != "US-SUPPRESSED (SEE DATA DEFINITIONS)"
uslocations = locations[mask1 & mask2]
states = uslocations["State"]

#the split will split by the comma, the get will get the position after the thing we just split
#the title will put in title rather than all caps, and the value counts determines the values. 

state_totals = states.str.split("-").str.get(1).str.title().value_counts()
df = state_totals.rename_axis('State').reset_index(name='Total Employees')
df

,State,Total Employees
0,California,143701
1,District Of Columbia,141514
2,Virginia,135656
3,Maryland,129234
4,Texas,116691
5,Florida,79771
6,Georgia,72079
7,Pennsylvania,59861
8,Washington,54918
9,New York,51676


In [6]:
df['Longitude'] = ""
df['Latitude'] = ""
df.head()

,State,Total Employees,Longitude,Latitude
0,California,143701,,
1,District Of Columbia,141514,,
2,Virginia,135656,,
3,Maryland,129234,,
4,Texas,116691,,


In [17]:
for index, row in df.iterrows():
    
    #this will get the state for the current row
    target_location = df.at[index, 'State']
    
    #you should also be able to do this    
    target_location = row['State']

    #now make the API call so that your target location will be the state for each time you loop through
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}').format(target_location, gkey)
    
    #assemble url and make API request
    #set this before trying to use it below 
    geo_data = requests.get(target_url).json()

    # Extract latitude and longitude
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]

    #pprint(geo_data)

    #loc or at should work fine
    df.at[index, "Latitude"] = lat
    df.at[index, "Longitude"] = lng

df.dtypes

State              object
Total Employees     int64
Longitude          object
Latitude           object
dtype: object

In [30]:
#store long and lat
locations = df[["Latitude", "Longitude"]]

#store total employees and change value to float
state_employees = df["Total Employees"].astype(float)

In [31]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=state_employees, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=state_employees, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))